In [38]:
import requests
import json
import subprocess
import sys
import pandas as pd

from IPython.core.display import display,HTML


from requests import session


In [39]:
# authorize_url = api_key.secret.get('AUTHORIZE')
# callback_uri = api_key.secret.get('CALL_BACK_URL')
# app_id = api_key.secret.get('APPLICATION_ID')
# app_secret = api_key.secret.get('SECRET')

callback_uri = api_key.secret.get('CALL_BACK_URL')
app_id = api_key.secret.get('APPLICATION_ID')
app_secret = api_key.secret.get('SECRET')
# user_name = api_key.secret.get('USER_NAME')
# password = api_key.secret.get('PASSWORD')

site = "https://www.inaturalist.org"
token_url = "https://www.inaturalist.org/users/api_token"
test_api_url = "https://api.inaturalist.org/"

# print(authorize_url)


In [3]:
# generate authenticate url
url = site + '/oauth/authorize?client_id=' + app_id + '&redirect_uri=' + callback_uri + '&response_type=code'
auth_response = requests.get(url)

data = {'grant_type': 'authorization_code', 'redirect_uri': callback_uri}

payload = {
'grant_type': "client_credentials"
}


{'grant_type': 'authorization_code', 'redirect_uri': 'https://github.com/ARSimmons/red_cedar_data'}


In [4]:
# get access token from app_id, secret key
access_token_response = requests.post(site + '/oauth/token', data=payload, verify=False, allow_redirects=False,
auth=(app_id, app_secret))

tokens = json.loads(access_token_response.text)['access_token']
token_age = json.loads(access_token_response.text)['created_at']
print(tokens)
print(token_age)


I:\Anaconda\envs\dash\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.inaturalist.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


fcfdd7e0c4a5c4ccc95e9aba487aab8ab771c06ab57ed375f6e72216596207b8
1615613457


In [36]:
# get observation
# payload = {
#     'p': 'quality_grade=any&identifications=any&projects%5B%5D=red_cedar_data',
# }

# fetch oberservation data from observation api (/observations.json)
page = 1
json_data = []
while True:
    payload = {    
        'q': 'Western Redcedar',     # search string    
#         'taxon_name': ['Thuja plicata'], # taxon names    
        'page': page, # page number from 1 to any number
        'per_page': 200 # observation count for each page
    }

    data = requests.get(site + '/observations.json?',params=payload, headers={'Authorization': tokens})
    sub_data = data.json()
    
    if len(sub_data) < 1: # break to fetch data if there is no data in current page
        break
        
    json_data = json_data + sub_data
    page = page + 1 # increase page number
    if page > 3: # 2
        break
    

print("JSON Response =================================================================================")
print("Data Length = ", str(len(json_data))) # display the number of observation

# Change data
for row in json_data:
    # parse taxon    
    if 'taxon' in row: # extract attributes from taxon key
        taxon = row['taxon']

        row['taxon_id'] = taxon['id']
        row['taxon_name'] = taxon['name']
        row['rank'] = taxon['rank']
        row['ancestry'] = taxon['ancestry']
        
        if 'common_name' in taxon and taxon['common_name']:
            row['common_name'] = taxon['common_name']['name']
            row['common_name_id'] = taxon['common_name']['id']
            row['common_name_name'] = taxon['common_name']['name']
            row['is_valid'] = taxon['common_name']['is_valid']
            row['lexicon'] = taxon['common_name']['lexicon']

        del row['taxon']
        
    # parse iconic_taxon
    if 'iconic_taxon' in row: # extract attrbutes from iconic_taxon
        iconic_taxon  = row['iconic_taxon']
        
        row['iconic_taxon_name'] = iconic_taxon['name']
        row['rank_level'] = iconic_taxon['rank_level']
        
        del row['iconic_taxon']   
    
    
    photos = row['photos'] # get photo url
    if len(photos) > 0:
        row['medium_photo_url'] = photos[0]['medium_url']
    del row['photos']
    
print(json_data)

JSON Response =================================================================================
Data Length =  600
[{'id': 71114016, 'observed_on': '2021-03-12', 'description': None, 'latitude': '36.48953667', 'longitude': '-121.19725833', 'map_scale': None, 'timeframe': None, 'species_guess': None, 'user_id': 2563702, 'taxon_id': 49651, 'created_at': '2021-03-13T06:00:18.173Z', 'updated_at': '2021-03-13T06:00:18.219Z', 'place_guess': 'Paicines', 'id_please': False, 'observed_on_string': '2021-03-12 12:53:25-08:00', 'iconic_taxon_id': 47126, 'num_identification_agreements': 0, 'num_identification_disagreements': 0, 'time_observed_at': '2021-03-12T20:53:25.000Z', 'time_zone': 'Pacific Time (US & Canada)', 'location_is_exact': False, 'delta': False, 'positional_accuracy': None, 'private_latitude': None, 'private_longitude': None, 'geoprivacy': None, 'quality_grade': 'casual', 'positioning_method': None, 'positioning_device': None, 'out_of_range': None, 'license': None, 'uri': 'https://ww

In [26]:
# generate panda frames from dictionary list
df = pd.DataFrame.from_dict(json_data) 

print("Parsed CSV Data =================================================================================\n")
# print(df.head(5))

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)
# display table data with html table format (including image)
display(HTML(df.to_html(escape=False ,formatters=dict(medium_photo_url=path_to_image_html))))


Parsed CSV Data =================================================================================



,id,observed_on,description,latitude,longitude,map_scale,timeframe,species_guess,user_id,taxon_id,created_at,updated_at,place_guess,id_please,observed_on_string,iconic_taxon_id,num_identification_agreements,num_identification_disagreements,time_observed_at,time_zone,location_is_exact,delta,positional_accuracy,private_latitude,private_longitude,geoprivacy,quality_grade,positioning_method,positioning_device,out_of_range,license,uri,observation_photos_count,comments_count,zic_time_zone,oauth_application_id,observation_sounds_count,identifications_count,captive,community_taxon_id,site_id,old_uuid,public_positional_accuracy,mappable,cached_votes_total,last_indexed_at,private_place_guess,uuid,taxon_geoprivacy,short_description,user_login,iconic_taxon_name,tag_list,faves_count,created_at_utc,updated_at_utc,time_observed_at_utc,owners_identification_from_vision,user,taxon_name,rank,ancestry,common_name,common_name_id,common_name_name,is_valid,lexicon,rank_level,medium_photo_url,coordinates_obscured
0,71093987,2021-03-10,,48.4942164559,-123.3473911514,NaN,None,,3945649,48252,2021-03-12T23:34:33.642Z,2021-03-12T23:35:10.911Z,"Blenkinsop, Victoria, BC, CA",False,Wed Mar 10 2021 12:00:00 GMT-0800 (PST),47126,0,0,2021-03-10T12:00:00.000Z,UTC,False,False,2162.0,NaN,NaN,None,needs_id,None,None,None,CC-BY-NC,https://www.inaturalist.org/observations/71093987,4,0,Etc/UTC,3.0,0,1,False,NaN,1.0,None,2162.0,True,0,2021-03-13T02:54:21.551Z,NaN,0f3970e8-67be-4007-9dc4-b78646e92426,open,,harkiratchohan,Plantae,[],0,2021-03-12T23:34:33.642Z,2021-03-12T23:35:10.911Z,2021-03-10T12:00:00.000Z,True,"{'login': 'harkiratchohan', 'user_icon_url': None}",Thuja plicata,species,48460/47126/211194/136329/47375/47374/1150915/48253,western redcedar,77887,western redcedar,True,English,70.0,,NaN
1,71090540,2021-03-12,,49.0827498545,-117.801981711,NaN,None,western redcedar,3953583,48252,2021-03-12T22:39:34.725Z,2021-03-12T22:58:43.109Z,"Rossland, BC, Canada",False,2021/03/12 2:04 PM UTC,47126,1,0,2021-03-12T14:04:00.000Z,UTC,False,False,1.0,NaN,NaN,None,research,None,None,None,CC-BY-NC,https://www.inaturalist.org/observations/71090540,2,0,Etc/UTC,NaN,0,2,False,48252.0,1.0,None,1.0,True,0,2021-03-12T22:58:47.650Z,NaN,ae9218a3-bfe1-4b9a-a1fb-b9c801b5df93,open,,connjulia,Plantae,[],0,2021-03-12T22:39:34.725Z,2021-03-12T22:58:43.109Z,2021-03-12T14:04:00.000Z,True,"{'login': 'connjulia', 'user_icon_url': None}",Thuja plicata,species,48460/47126/211194/136329/47375/47374/1150915/48253,western redcedar,77887,western redcedar,True,English,70.0,,NaN
2,71090538,2021-03-12,,49.0827523438,-117.8020976631,NaN,None,Western Redcedar,3953583,48252,2021-03-12T22:39:34.087Z,2021-03-12T23:00:23.965Z,"Rossland, BC, Canada",False,2021/03/12 2:03 PM UTC,47126,0,0,2021-03-12T14:03:00.000Z,UTC,False,False,1.0,NaN,NaN,None,needs_id,None,None,None,CC-BY-NC,https://www.inaturalist.org/observations/71090538,3,1,Etc/UTC,NaN,0,1,False,NaN,1.0,None,1.0,True,0,2021-03-12T23:00:26.965Z,NaN,ed162af0-4703-4042-bbd2-24dc11b7713a,open,,connjulia,Plantae,[],0,2021-03-12T22:39:34.087Z,2021-03-12T23:00:23.965Z,2021-03-12T14:03:00.000Z,True,"{'login': 'connjulia', 'user_icon_url': None}",Thuja plicata,species,48460/47126/211194/136329/47375/47374/1150915/48253,western redcedar,77887,western redcedar,True,English,70.0,,NaN
3,71090537,2021-03-12,,49.082780452,-117.802019772,NaN,None,western redcedar,3953583,48252,2021-03-12T22:39:33.954Z,2021-03-12T23:00:03.317Z,"Rossland, BC, Canada",False,2021/03/12 1:58 PM UTC,47126,1,0,2021-03-12T13:58:00.000Z,UTC,False,False,1.0,NaN,NaN,None,research,None,None,None,CC-BY-NC,https://www.inaturalist.org/observations/71090537,2,1,Etc/UTC,NaN,0,2,False,48252.0,1.0,None,1.0,True,0,2021-03-12T23:00:10.330Z,NaN,683c9888-4b99-4d5b-8f49-a8e491d657a0,open,,connjulia,Plantae,[],0,2021-03-12T22:39:33.954Z,2021-03-12T23:00:03.317Z,2021-03-12T13:58:00.000Z,True,"{'login': 'connjulia', 'user_icon_url': None}",Thuja plicata,species,48460/47126/211194/136329/47375/47374/1150915/48253,wes

In [37]:
# save csv
df.to_csv('observation.csv', encoding='utf-8')